In [1]:
##setting work directory for R
mydir<- getwd()
setwd(mydir)

In [ ]:
##setting work directory for bash
%use bash
mydir=`pwd`
cd $mydir

In [ ]:
##loading R libraries
.libPaths(c("/anvil/projects/x-tra220018/2022/Rlibs", .libPaths()))
library(BiocParallel)
library(dplyr)
library(EnsDb.Hsapiens.v86)
library(GenomicAlignments)
library(GenomicRanges) 
library(Gviz)
library(parallel)
library(Rsamtools)
library(ShortRead)

In [ ]:
#checking quality of single-end data
#%use bash
module load fastqc/0.11.9
data_path=/home/x-tsuzuki/bigcare/myproject/raw
fastqc -t 20 $data_path/*.fastq.gz -o ./data/fastqc/raw/

module load multiqc
data_path=/home/x-tsuzuki/bigcare/myproject/data/fastqc/raw
multiqc $data_path -o $data_path

In [ ]:
# trimming adapters from reads
# PDX4_CR1_S7_R1_001
module load trimmomatic/0.39
data_path=/home/x-tsuzuki/bigcare/myproject/raw
out_path=./data/trim
trimmomatic SE -phred33 -threads 15 \
$data_path/PDX4_CR1_S7_R1_001.fastq.gz  \
$out_path/PDX4_CR1_S7_R1_001.fastq.gz \
ILLUMINACLIP:/home/x-tsuzuki/bigcare/ref_files/TruSeq2-SE.fa:2:30:10 \
LEADING:10 TRAILING:10 SLIDINGWINDOW:4:20 MINLEN:20

# PDX4_CR2_S8_R1_001
trimmomatic SE -phred33 -threads 15 \
$data_path/PDX4_CR2_S8_R1_001.fastq.gz  \
$out_path/PDX4_CR2_S8_R1_001.fastq.gz \
ILLUMINACLIP:/home/x-tsuzuki/bigcare/ref_files/TruSeq2-SE.fa:2:30:10 \
LEADING:10 TRAILING:10 SLIDINGWINDOW:4:20 MINLEN:20

# PDX4_CR3_S9_R1_001
trimmomatic SE -phred33 \
$data_path/PDX4_CR3_S9_R1_001.fastq.gz  \
$out_path/PDX4_CR3_S9_R1_001.fastq.gz \
ILLUMINACLIP:/home/x-tsuzuki/bigcare/ref_files/TruSeq2-SE.fa:2:30:10 \
LEADING:10 TRAILING:10 SLIDINGWINDOW:4:20 MINLEN:20

# PDX4_SE1_S10_R1_001
trimmomatic SE -phred33 -threads 15 \
$data_path/PDX4_SE1_S10_R1_001.fastq.gz \
$out_path/PDX4_SE1_S10_R1_001.fastq.gz \
ILLUMINACLIP:/home/x-tsuzuki/bigcare/ref_files/TruSeq2-SE.fa:2:30:10 \
LEADING:10 TRAILING:10 SLIDINGWINDOW:4:20 MINLEN:20

# PDX4_SE2_S11_R1_001
trimmomatic SE -phred33 -threads 15 \
$data_path/PDX4_SE2_S11_R1_001.fastq.gz \
$out_path/PDX4_SE2_S11_R1_001.fastq.gz \
ILLUMINACLIP:/home/x-tsuzuki/bigcare/ref_files/TruSeq2-SE.fa:2:30:10 \
LEADING:10 TRAILING:10 SLIDINGWINDOW:4:20 MINLEN:20

# PDX4_SE3_S12_R1_001
trimmomatic SE -phred33 -threads 15 \
$data_path/PDX4_SE3_S12_R1_001.fastq.gz \
$out_path/PDX4_SE3_S12_R1_001.fastq.gz \
ILLUMINACLIP:/home/x-tsuzuki/bigcare/ref_files/TruSeq2-SE.fa:2:30:10 \
LEADING:10 TRAILING:10 SLIDINGWINDOW:4:20 MINLEN:20

In [ ]:
#quality control after trimming single-end
module load fastqc/0.11.9
data_path=/home/x-tsuzuki/bigcare/myproject/data
fastqc -t 20 $data_path/trim/*.fastq.gz -o ./data/fastqc/trim/

In [ ]:
#the following steps were based on GeneLab's pipeline
#generating genome index for ensembl annotations
#release-109
#ensembl
conda activate apps
module load star/2.7.10a
STAR --runThreadN 20 --runMode genomeGenerate \
--genomeDir /home/x-tsuzuki/bigcare/ref_files/genomeassembly \
--genomeFastaFiles /home/x-tsuzuki/Homo_sapiens.GRCh38.dna.primary_assembly.fa \
--sjdbGTFfile /home/x-tsuzuki/Homo_sapiens.GRCh38.109.gtf \
--sjdbOverhang 75

In [ ]:
##aligning single-end data to genome
# PDX4_CR1_S7_R1_001
module load star/2.7.10a
data_path=./data/trim
out_path=./data/STAR/single
STAR --twopassMode Basic \
--genomeDir /home/x-tsuzuki/bigcare/ref_files/genomeassembly \
--outSAMunmapped Within \
--outFilterType BySJout \
--outSAMattributes NH HI AS NM MD MC \
--outFilterMultimapNmax 20 \
--outFilterMismatchNmax 999 \
--outFilterMismatchNoverReadLmax 0.04 \
--alignIntronMin 20 \
--alignIntronMax 1000000 \
--alignSJoverhangMin 8 \
--alignSJDBoverhangMin 1 \
--sjdbScore 1 \
--readFilesCommand zcat \
--runThreadN 20 \
--outSAMtype BAM SortedByCoordinate \
--quantMode TranscriptomeSAM GeneCounts \
--outSAMheaderHD @HD VN:1.4 SO:coordinate \
--outFileNamePrefix $out_path/PDX4_CR1_ \
--readFilesIn $data_path/PDX4_CR1_S7_R1_001.fastq.gz

# PDX4_CR2_S8_R1_001
STAR --twopassMode Basic \
--genomeDir /home/x-tsuzuki/bigcare/ref_files/genomeassembly \
--outSAMunmapped Within \
--outFilterType BySJout \
--outSAMattributes NH HI AS NM MD MC \
--outFilterMultimapNmax 20 \
--outFilterMismatchNmax 999 \
--outFilterMismatchNoverReadLmax 0.04 \
--alignIntronMin 20 \
--alignIntronMax 1000000 \
--alignSJoverhangMin 8 \
--alignSJDBoverhangMin 1 \
--sjdbScore 1 \
--readFilesCommand zcat \
--runThreadN 20 \
--outSAMtype BAM SortedByCoordinate \
--quantMode TranscriptomeSAM GeneCounts \
--outSAMheaderHD @HD VN:1.4 SO:coordinate \
--outFileNamePrefix $out_path/PDX4_CR2_ \
--readFilesIn $data_path/PDX4_CR2_S8_R1_001.fastq.gz

# PDX4_CR3_S9_R1_001
STAR --twopassMode Basic \
--genomeDir /home/x-tsuzuki/bigcare/ref_files/genomeassembly \
--outSAMunmapped Within \
--outFilterType BySJout \
--outSAMattributes NH HI AS NM MD MC \
--outFilterMultimapNmax 20 \
--outFilterMismatchNmax 999 \
--outFilterMismatchNoverReadLmax 0.04 \
--alignIntronMin 20 \
--alignIntronMax 1000000 \
--alignSJoverhangMin 8 \
--alignSJDBoverhangMin 1 \
--sjdbScore 1 \
--readFilesCommand zcat \
--runThreadN 20 \
--outSAMtype BAM SortedByCoordinate \
--quantMode TranscriptomeSAM GeneCounts \
--outSAMheaderHD @HD VN:1.4 SO:coordinate \
--outFileNamePrefix $out_path/PDX4_CR3_ \
--readFilesIn $data_path/PDX4_CR3_S9_R1_001.fastq.gz

# PDX4_SE1_S10_R1_001
STAR --twopassMode Basic \
--genomeDir /home/x-tsuzuki/bigcare/ref_files/genomeassembly \
--outSAMunmapped Within \
--outFilterType BySJout \
--outSAMattributes NH HI AS NM MD MC \
--outFilterMultimapNmax 20 \
--outFilterMismatchNmax 999 \
--outFilterMismatchNoverReadLmax 0.04 \
--alignIntronMin 20 \
--alignIntronMax 1000000 \
--alignSJoverhangMin 8 \
--alignSJDBoverhangMin 1 \
--sjdbScore 1 \
--readFilesCommand zcat \
--runThreadN 20 \
--outSAMtype BAM SortedByCoordinate \
--quantMode TranscriptomeSAM GeneCounts \
--outSAMheaderHD @HD VN:1.4 SO:coordinate \
--outFileNamePrefix $out_path/PDX4_SE1_ \
--readFilesIn $data_path/PDX4_SE1_S10_R1_001.fastq.gz

# PDX4_SE2_S11_R1_001
STAR --twopassMode Basic \
--genomeDir /home/x-tsuzuki/bigcare/ref_files/genomeassembly \
--outSAMunmapped Within \
--outFilterType BySJout \
--outSAMattributes NH HI AS NM MD MC \
--outFilterMultimapNmax 20 \
--outFilterMismatchNmax 999 \
--outFilterMismatchNoverReadLmax 0.04 \
--alignIntronMin 20 \
--alignIntronMax 1000000 \
--alignSJoverhangMin 8 \
--alignSJDBoverhangMin 1 \
--sjdbScore 1 \
--readFilesCommand zcat \
--runThreadN 20 \
--outSAMtype BAM SortedByCoordinate \
--quantMode TranscriptomeSAM GeneCounts \
--outSAMheaderHD @HD VN:1.4 SO:coordinate \
--outFileNamePrefix $out_path/PDX4_SE2_ \
--readFilesIn $data_path/PDX4_SE2_S11_R1_001.fastq.gz

# PDX4_SE3_S12_R1_001
STAR --twopassMode Basic \
--genomeDir /home/x-tsuzuki/bigcare/ref_files/genomeassembly \
--outSAMunmapped Within \
--outFilterType BySJout \
--outSAMattributes NH HI AS NM MD MC \
--outFilterMultimapNmax 20 \
--outFilterMismatchNmax 999 \
--outFilterMismatchNoverReadLmax 0.04 \
--alignIntronMin 20 \
--alignIntronMax 1000000 \
--alignSJoverhangMin 8 \
--alignSJDBoverhangMin 1 \
--sjdbScore 1 \
--readFilesCommand zcat \
--runThreadN 20 \
--outSAMtype BAM SortedByCoordinate \
--quantMode TranscriptomeSAM GeneCounts \
--outSAMheaderHD @HD VN:1.4 SO:coordinate \
--outFileNamePrefix $out_path/PDX4_SE3_ \
--readFilesIn $data_path/PDX4_SE3_S12_R1_001.fastq.gz

In [ ]:
## sort aligned reads
conda activate apps
data_path=/home/x-tsuzuki/bigcare/myproject/data/STAR/single
samtools sort -m 3G --threads 20 -o $data_path/PDX4_CR1_Aligned.sortedByCoord.out.bam $data_path/PDX4_CR1_Aligned.sortedByCoord.out.bam
samtools sort -m 3G --threads 20 -o $data_path/PDX4_CR2_Aligned.sortedByCoord.out.bam $data_path/PDX4_CR2_Aligned.sortedByCoord.out.bam
samtools sort -m 3G --threads 20 -o $data_path/PDX4_CR3_Aligned.sortedByCoord.out.bam $data_path/PDX4_CR3_Aligned.sortedByCoord.out.bam
samtools sort -m 3G --threads 20 -o $data_path/PDX4_SE1_Aligned.sortedByCoord.out.bam $data_path/PDX4_SE1_Aligned.sortedByCoord.out.bam
samtools sort -m 3G --threads 20 -o $data_path/PDX4_SE2_Aligned.sortedByCoord.out.bam $data_path/PDX4_SE2_Aligned.sortedByCoord.out.bam
samtools sort -m 3G --threads 20 -o $data_path/PDX4_SE3_Aligned.sortedByCoord.out.bam $data_path/PDX4_SE3_Aligned.sortedByCoord.out.bam

In [ ]:
## indexing alignments 
module load samtools/1.12
data_path=/home/x-tsuzuki/bigcare/myproject/data/STAR/single
samtools index -@ 20 $data_path/PDX4_CR1_Aligned.sortedByCoord.out.bam 
samtools index -@ 20 $data_path/PDX4_CR2_Aligned.sortedByCoord.out.bam 
samtools index -@ 20 $data_path/PDX4_CR3_Aligned.sortedByCoord.out.bam
samtools index -@ 20 $data_path/PDX4_SE1_Aligned.sortedByCoord.out.bam
samtools index -@ 20 $data_path/PDX4_SE2_Aligned.sortedByCoord.out.bam
samtools index -@ 20 $data_path/PDX4_SE3_Aligned.sortedByCoord.out.bam

In [ ]:
#reference for RSEM
rsem-prepare-reference --gtf /home/x-tsuzuki/Homo_sapiens.GRCh38.109.gtf \
/home/x-tsuzuki/Homo_sapiens.GRCh38.dna.primary_assembly.fa \
/home/x-tsuzuki/bigcare/ref_files/RSEM/RSEM_ref_prefix

In [ ]:
#counting expression with RSEM
#PDX4_CR1
rsem-calculate-expression --num-threads 20 \
--alignments \
--bam \
--seed 12345 \
--seed-length 20 \
--estimate-rspd \
--no-bam-output \
/home/x-tsuzuki/bigcare/myproject/data/STAR/single/PDX4_CR1_Aligned.toTranscriptome.out.bam \
/home/x-tsuzuki/bigcare/ref_files/RSEM/RSEM_ref_prefix \
/home/x-tsuzuki/bigcare/myproject/data/RSEM/PDX4_CR1

#PDX4_CR2
rsem-calculate-expression --num-threads 20 \
--alignments \
--bam \
--seed 12345 \
--seed-length 20 \
--estimate-rspd \
--no-bam-output \
/home/x-tsuzuki/bigcare/myproject/data/STAR/single/PDX4_CR2_Aligned.toTranscriptome.out.bam \
/home/x-tsuzuki/bigcare/ref_files/RSEM/RSEM_ref_prefix \
/home/x-tsuzuki/bigcare/myproject/data/RSEM/PDX4_CR2

#PDX4_CR3
rsem-calculate-expression --num-threads 20 \
--alignments \
--bam \
--seed 12345 \
--seed-length 20 \
--estimate-rspd \
--no-bam-output \
/home/x-tsuzuki/bigcare/myproject/data/STAR/single/PDX4_CR3_Aligned.toTranscriptome.out.bam \
/home/x-tsuzuki/bigcare/ref_files/RSEM/RSEM_ref_prefix \
/home/x-tsuzuki/bigcare/myproject/data/RSEM/PDX4_CR3

#PDX4_SE1
rsem-calculate-expression --num-threads 20 \
--alignments \
--bam \
--seed 12345 \
--seed-length 20 \
--estimate-rspd \
--no-bam-output \
/home/x-tsuzuki/bigcare/myproject/data/STAR/single/PDX4_SE1_Aligned.toTranscriptome.out.bam \
/home/x-tsuzuki/bigcare/ref_files/RSEM/RSEM_ref_prefix \
/home/x-tsuzuki/bigcare/myproject/data/RSEM/PDX4_SE1

#PDX4_SE2
rsem-calculate-expression --num-threads 20 \
--alignments \
--bam \
--seed 12345 \
--seed-length 20 \
--estimate-rspd \
--no-bam-output \
/home/x-tsuzuki/bigcare/myproject/data/STAR/single/PDX4_SE2_Aligned.toTranscriptome.out.bam \
/home/x-tsuzuki/bigcare/ref_files/RSEM/RSEM_ref_prefix \
/home/x-tsuzuki/bigcare/myproject/data/RSEM/PDX4_SE2

#PDX4_SE3
rsem-calculate-expression --num-threads 20 \
--alignments \
--bam \
--seed 12345 \
--seed-length 20 \
--estimate-rspd \
--no-bam-output \
/home/x-tsuzuki/bigcare/myproject/data/STAR/single/PDX4_SE3_Aligned.toTranscriptome.out.bam \
/home/x-tsuzuki/bigcare/ref_files/RSEM/RSEM_ref_prefix \
/home/x-tsuzuki/bigcare/myproject/data/RSEM/PDX4_SE3

In [ ]:
.libPaths(c("/home/x-tsuzuki/bigcare/Rlibs", .libPaths()))
library(tximport)
library(DESeq2)
library(tidyverse)
library(stringr)

In [ ]:
work_dir="/home/x-tsuzuki/bigcare/myproject"
counts_dir="/home/x-tsuzuki/bigcare/myproject/data/RSEM"

### Pull in sample names where the "samples.txt" file is a single column list of sample names ###
samples <- read.csv(Sys.glob(file.path(work_dir,"samples.txt")), header = FALSE, row.names = 1, stringsAsFactors = TRUE)

##### Import RSEM Gene Count Data
files <- list.files(file.path(counts_dir),pattern = ".genes.results", full.names = TRUE)

### reorder the genes.results files to match the ordering of the samples in the metadata file
files <- files[sapply(rownames(samples), function(x)grep(paste0(counts_dir, '/',  x,'.genes.results$'), files, value=FALSE))]

names(files) <- rownames(samples)
txi.rsem <- tximport(files, type = "rsem", txIn = FALSE, txOut = FALSE)

##### Count the number of genes with non-zero counts for each sample 
rawCounts <- txi.rsem$counts
NumNonZeroGenes <- (as.matrix(colSums(rawCounts > 0), row.names = 1))
colnames(NumNonZeroGenes) <- c("Number of genes with non-zero counts")

##### Export the number of genes with non-zero counts for each sample
setwd(file.path(counts_dir))
write.csv(NumNonZeroGenes,file='NumNonZeroGenes.csv')

## print session info ##
print("Session Info below: ")
print("")
sessionInfo()

In [27]:
organism <- "Homo sapiens"

runsheet_path="/home/x-tsuzuki/bigcare/myproject/PDX4_bulkRNASeq_v1_runsheet.csv"
work_dir="/home/x-tsuzuki/bigcare/myproject"
counts_dir="/home/x-tsuzuki/bigcare/myproject/data/RSEM"
norm_output="/home/x-tsuzuki/bigcare/myproject/data/RSEM/norm"
DGE_output="/home/x-tsuzuki/bigcare/myproject/data/RSEM/DGE"

### Pull in theannotation table (annotations.csv) file ###

org_table_link <- "/home/x-tsuzuki/bigcare/ref_files/hg38_index/annotations.csv"

org_table <- read.table(org_table_link, sep = ",", header = TRUE)

### Define the link to the GeneLab annotation table for the organism of interest ###

annotations_link <- org_table[org_table$name == organism, "genelab_annots_link"]


setwd(file.path(work_dir))

In [ ]:
### Pull all factors for each sample in the study from the runsheet created in Step 9a ###

compare_csv_from_runsheet <- function(runsheet_path) {
    df = read.csv(runsheet_path)
    # get only Factor Value columns
    factors = as.data.frame(df[,grep("Factor.Value", colnames(df), ignore.case=TRUE)])
    colnames(factors) = paste("factor",1:dim(factors)[2], sep= "_")
    result = data.frame(sample_id = df[,c("Sample.Name")], factors)	
    return(result)
}


### Load metadata from runsheet csv file ###

compare_csv <- compare_csv_from_runsheet(runsheet_path)


### Create data frame containing all samples and respective factors ###

study <- as.data.frame(compare_csv[,2:dim(compare_csv)[2]])
colnames(study) <- colnames(compare_csv)[2:dim(compare_csv)[2]]
rownames(study) <- compare_csv[,1]


### Format groups and indicate the group that each sample belongs to ###

if (dim(study) >= 2){
	group<-apply(study,1,paste,collapse = " & ") ## concatenate multiple factors into one condition per sample
} else{
	group<-study[,1]
}
group_names <- paste0("(",group,")",sep = "") ## human readable group names
group <- sub("^BLOCKER_", "",  make.names(paste0("BLOCKER_", group))) # group naming compatible with R models, this maintains the default behaviour of make.names with the exception that 'X' is never prepended to group names
names(group) <- group_names
rm(group_names)


### Format contrasts table, defining pairwise comparisons for all groups ###

contrast.names <- combn(levels(factor(names(group))),2) ## generate matrix of pairwise group combinations for comparison
contrasts <- apply(contrast.names, MARGIN=2, function(col) sub("^BLOCKER_", "",  make.names(paste0("BLOCKER_", stringr::str_sub(col, 2, -2))))) # limited make.names call for each group (also removes leading parentheses)
contrast.names <- c(paste(contrast.names[1,],contrast.names[2,],sep = "v"),paste(contrast.names[2,],contrast.names[1,],sep = "v")) ## format combinations for output table files names
contrasts <- cbind(contrasts,contrasts[c(2,1),])
colnames(contrasts) <- contrast.names
rm(contrast.names)  

In [ ]:
### Import RSEM raw (gene) count data ###

files <- list.files(file.path(counts_dir),pattern = ".genes.results", full.names = TRUE)


### Reorder the *genes.results files to match the ordering of the ISA samples ###

files <- files[sapply(rownames(study), function(x)grep(paste0(counts_dir, '/', x,".genes.results$"), files, value=FALSE))]

names(files) <- rownames(study)
txi.rsem <- tximport(files, type = "rsem", txIn = FALSE, txOut = FALSE)


### Add 1 to genes with lengths of zero - needed to make DESeqDataSet object ###

txi.rsem$length[txi.rsem$length == 0] <- 1


In [ ]:
### Create data frame defining which group each sample belongs to ###

sampleTable <- data.frame(condition=factor(group))
rownames(sampleTable) <- colnames(txi.rsem$counts)


### Make DESeqDataSet object ###

dds <- DESeqDataSetFromTximport(txi.rsem, sampleTable, ~condition)
summary(dds)


### Filter out genes with counts of less than 10 in all samples ###

keepGenes <- rowSums(counts(dds)) > 10
dds_1 <- dds[keepGenes,]
summary(dds_1)
dim(dds_1)


### Run DESeq analysis ###

dds_1 <- DESeq(dds_1)


### Generate F statistic p-value (similar to ANOVA p-value) using DESeq2 likelihood ratio test (LRT) design ###

dds_1_lrt <- DESeq(dds_1, test = "LRT", reduced = ~ 1)
res_1_lrt <- results(dds_1_lrt)

In [33]:
### Create a data frame containing normalized counts ###

normCounts <- as.data.frame(counts(dds_1, normalized=TRUE))

### Add 1 to all normalized counts to avoid issues with downstream calculations ###
normCounts <- normCounts +1


### Start the DGE output table with the normalized counts for all samples ###
## reduced output table 1 will be used to generate human-readable DGE table

reduced_output_table_1 <- normCounts

## output tables 1 will be used to generate computer-readable DGE table, which is used to create GeneLab visualization plots

output_table_1 <- normCounts

In [34]:
### Iterate through Wald Tests to generate pairwise comparisons of all groups ###

for (i in 1:dim(contrasts)[2]){
	res_1 <- results(dds_1, contrast=c("condition",contrasts[1,i],contrasts[2,i]))
	res_1 <- as.data.frame(res_1@listData)[,c(2,4,5,6)]
	colnames(res_1) <- c(paste0("Log2fc_", colnames(contrasts)[i]), paste0("Stat_",colnames(contrasts)[i]), paste0("P.value_",colnames(contrasts)[i]), paste0("Adj.p.value_",colnames(contrasts)[i]))
	output_table_1 <- cbind(output_table_1,res_1)
	reduced_output_table_1 <- cbind(reduced_output_table_1,res_1)
	rm(res_1)
}

In [35]:
### Generate and add all sample mean column to the DGE table ###

output_table_1$All.mean <- rowMeans(normCounts, na.rm = TRUE, dims = 1)
reduced_output_table_1$All.mean <- rowMeans(normCounts, na.rm = TRUE, dims = 1)

### Generate and add all sample stdev column to the DGE table ###

output_table_1$All.stdev <- rowSds(as.matrix(normCounts), na.rm = TRUE, dims = 1)
reduced_output_table_1$All.stdev <- rowSds(as.matrix(normCounts), na.rm = TRUE, dims = 1)

In [36]:
### Add F statistic p-value (similar to ANOVA p-value) column to the DGE table ###

output_table_1$LRT.p.value <- res_1_lrt@listData$padj
reduced_output_table_1$LRT.p.value <- res_1_lrt@listData$padj

In [37]:
### Generate and add group mean and stdev columns to the DGE table ###

tcounts <- as.data.frame(t(normCounts))
tcounts$group <- names(group) # Used final table group name formatting (e.g. '( Space Flight & Blue Light )' )

In [ ]:
library(BiocParallel)
library(parallel)
bpparam <- MulticoreParam(workers=20, tasks=0)
register(bpparam)

registered()$MulticoreParam

group_means <- as.data.frame(t(aggregate(. ~ group,data = tcounts,mean))) # Compute group name group-wise means
colnames(group_means) <- paste0("Group.Mean_", group_means['group',]) # assign group name as column names

In [39]:

group_stdev <- as.data.frame(t(aggregate(. ~ group,data = tcounts,sd))) # Compute group name group-wise standard deviation
colnames(group_stdev) <- paste0("Group.Stdev_", group_stdev['group',]) # assign group name as column names

In [40]:
group_means <- group_means[-c(1),] # Drop group name row from data rows (now present as column names)
group_stdev <- group_stdev[-c(1),] # Drop group name row from data rows (now present as column names)
output_table_1 <- cbind(output_table_1,group_means, group_stdev) # Column bind the group-wise data
reduced_output_table_1 <- cbind(reduced_output_table_1,group_means, group_stdev) # Column bind the group-wise data

rm(group_stdev,group_means,tcounts)

In [ ]:
### Add columns needed to generate GeneLab visualization plots to the DGE table ###

## Add column to indicate the sign (positive/negative) of log2fc for each pairwise comparison ##

updown_table <- sign(output_table_1[,grep("Log2fc_",colnames(output_table_1))])
colnames(updown_table) <- gsub("Log2fc","Updown",grep("Log2fc_",colnames(output_table_1),value = TRUE))
output_table_1 <- cbind(output_table_1,updown_table)
rm(updown_table)


## Add column to indicate contrast significance with p <= 0.1 ##

sig.1_table <- output_table_1[,grep("P.value_",colnames(output_table_1))]<=.1
colnames(sig.1_table) <- gsub("P.value","Sig.1",grep("P.value_",colnames(output_table_1),value = TRUE))
output_table_1 <- cbind(output_table_1,sig.1_table)
rm(sig.1_table)


## Add column to indicate contrast significance with p <= 0.05 ##

sig.05_table <- output_table_1[,grep("P.value_",colnames(output_table_1))]<=.05
colnames(sig.05_table) <- gsub("P.value","Sig.05",grep("P.value_",colnames(output_table_1),value = TRUE))
output_table_1 <- cbind(output_table_1,sig.05_table)
rm(sig.05_table)


## Add columns for the volcano plot with p-value and adjusted p-value ##

log_pval_table <- log2(output_table_1[,grep("P.value_",colnames(output_table_1))])
colnames(log_pval_table) <- paste0("Log2_",colnames(log_pval_table))
output_table_1 <- cbind(output_table_1,log_pval_table)
rm(log_pval_table)
log_adj_pval_table <- log2(output_table_1[,grep("Adj.p.value_",colnames(output_table_1))])
colnames(log_adj_pval_table) <- paste0("Log2_",colnames(log_adj_pval_table))
output_table_1 <- cbind(output_table_1,log_adj_pval_table)
rm(log_adj_pval_table)


## Prepare PCA table for GeneLab visualization plots ##

exp_raw <- log2(normCounts)
PCA_raw <- prcomp(t(exp_raw), scale = FALSE)

exp_raw
PCA_raw

In [42]:
### Read in GeneLab annotation table for the organism of interest ###
annot <- read.table("/home/x-tsuzuki/bigcare/ref_files/Homo_sapiens.GRCh38.109-GL-annotations.tsv", sep = "\t", header = TRUE, quote = "", comment.char = "", row.names = 1)

In [ ]:
### Export unnormalized and normalized counts tables ###

normCounts_exp <- as.data.frame(counts(dds_1, normalized=TRUE))

write.csv(txi.rsem$counts,file.path(norm_output, "RSEM_Unnormalized_Counts.csv"))
write.csv(normCounts_exp,file.path(norm_output, "Normalized_Counts.csv"))


### Export sample grouping and contrasts tables ###

write.csv(sampleTable,file.path(DGE_output, "SampleTable.csv"))

write.csv(contrasts,file.path(DGE_output, "contrasts.csv"))


### Export human-readable DGE table ###

write.csv(reduced_output_table_1,file.path(DGE_output, "differential_expression.csv"), row.names = FALSE)


### Export computer-readable DGE and PCA tables used for GeneLab visualization ###

write.csv(output_table_1,file.path(DGE_output, "visualization_output_table.csv"), row.names = FALSE)

write.csv(PCA_raw$x,file.path(DGE_output, "visualization_PCA_table.csv"), row.names = TRUE)


### print session info ###

print("Session Info below: ")
sessionInfo()